najmanji broj klastera za najvecim siluet skorom
siluet skor za svaki klaster i onda mean

In [ ]:
import pandas as pd
import matplotlib as plt
import numpy as np
import seaborn as sb
import multiprocessing as mp
import os
import cupy as cp
import cuml as cm
from matplotlib import pyplot as plt
import matplotlib.colors as mcolors
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import joblib

In [ ]:
data = pd.read_csv('./dataset.csv', index_col=0)

data.reset_index(inplace=True, drop=True)
data.head()

data['CellID'] = data['CellID'].astype(np.uint16)
data['ExonCount'] = data['ExonCount'].astype(np.uint8)
data['MIDCount'] = data['MIDCount'].astype(np.uint8)
data['x'] = data['x'].astype(np.uint16)
data['y'] = data['y'].astype(np.uint16)

unique_cells = data['CellID'].unique()
unique_genes = data['geneID'].unique()
print(f'{len(unique_cells)} unique cells and {len(unique_genes)} unique genes')

xmin, ymin = data['x'].min(), data['y'].min()
data['x'], data['y'] = data['x'] - xmin, data['y'] - ymin
xmax, ymax = data['x'].max(), data['y'].max()
xmax += 1
ymax += 1

In [ ]:
def generate_picture() -> np.ndarray:
    __data = data[['x', 'y']]
    picture = np.zeros(shape=(xmax, ymax), dtype=np.uint8)

    for row in range(__data.shape[0]):
        x, y = __data.iloc[row]
        picture[x][y] = 1

    plt.figure(figsize=(xmax/100, ymax/100))
    plt.imshow(picture, cmap='gray')
    plt.axis('off')
    plt.show()
    if not os.path.exists('cells.tiff'):
        plt.imsave('./cells.tiff', picture, cmap='gray')

    return picture

picture = generate_picture()

In [ ]:
def midcount_filling() -> int:
    gpu_matrix = cp.zeros((len(unique_cells), len(unique_genes)), dtype=cp.uint8)
    cell_indices = pd.factorize(data['CellID'])[0] 
    gene_indices = pd.factorize(data['geneID'])[0]

    gpu_cell_indices = cp.array(cell_indices, dtype=cp.uint16)
    gpu_gene_indices = cp.array(gene_indices, dtype=cp.uint16)
    gpu_midcount = cp.array(data['MIDCount'].values, dtype=cp.uint8)

    gpu_matrix[gpu_cell_indices, gpu_gene_indices] = gpu_midcount

    tmp_matrix = cp.asnumpy(gpu_matrix)
    print(tmp_matrix.shape) #####

    unloged_matrix = pd.DataFrame(tmp_matrix, index=unique_cells, columns=unique_genes, dtype=np.uint8)
    unloged_matrix.to_csv('unloged_matrix.csv', index=False)

    unloged_matrix.info()
    return unloged_matrix.last_valid_index()

unloged_matrix_number_of_rows = midcount_filling()

half_rows = unloged_matrix_number_of_rows // 2
second_half_rows = unloged_matrix_number_of_rows - half_rows

In [ ]:
def loging_matrix(data_chunk, mode) -> None:
    gpu_matrix = cp.array(data_chunk.values, dtype=cp.float32)

    gpu_matrix[gpu_matrix == 0] = -1
    gpu_matrix = cp.where(gpu_matrix == -1, -1, cp.log(gpu_matrix))

    tmp_matrix = cp.asnumpy(gpu_matrix)

    loged_matrix = pd.DataFrame(tmp_matrix, index=data_chunk.index, columns=data_chunk.columns, dtype=np.float32)
    loged_matrix.to_csv('loged_matrix.csv', mode=mode, header=(mode == 'w'), index=False)

def first_part_loging() -> None:
    first_half = pd.read_csv('unloged_matrix.csv', nrows=half_rows)
    loging_matrix(first_half, mode='w')

def second_part_loging() -> None:
    second_half = pd.read_csv('unloged_matrix.csv', skiprows=un, nrows=second_half_rows)
    loging_matrix(second_half, mode='a')

first_part_loging()
second_part_loging()

In [14]:
def loader(file_name : str) -> np.ndarray:
    tmp = pd.read_csv(file_name, index_col=0)
    tmp = np.array(tmp.values)
    return tmp

def clustering() -> any:
    X = loader('unloged_matrix.csv')
    model = KMeans(n_clusters=20)
    model.fit(X)
    joblib.dump(model, 'model.pkl')
    
    labels_df = pd.DataFrame(model.labels_)

    return model.labels_

labels = clustering()

In [ ]:
def generate_clustered_picture() -> None:
    __data = data[['x', 'y', 'CellID']]
    __labels = pd.DataFrame(labels, index=unique_cells)
    picture = np.zeros(shape=(xmax, ymax), dtype=np.uint8)

    for row in range(__data.shape[0]):
        x, y, cellid = __data.iloc[row]
        picture[x][y] = __labels.loc[cellid][0]

    cmap = plt.get_cmap('tab20', 20) ###
    tmp = list(cmap.colors)
    tmp[0] = (0, 0, 0) 
    cmap = mcolors.ListedColormap(tmp)
    
    plt.figure(figsize=(xmax/100, ymax/100))
    plt.imshow(picture, cmap=cmap)
    plt.axis('off')
    plt.colorbar(ticks=np.arange(20)) ###
    plt.show()
    plt.imsave('clustered_cells.tiff', picture, cmap=cmap)

    return picture

clustered_picture = generate_clustered_picture()


In [ ]:
def score(labels : list) -> None:
    data = loader('unloged_matrix.csv')
    score = silhouette_score(data, labels)
    print(score)

score(labels)